# Basic image processing using the LSST stack

The purpose of this demo is to process a single HSC image in order to produce a calibrated exposure (`calexp`) and a catalog of sources 

The HSC raw images has been imported from NCSA. They are located in `/sps/lsst/dev/lsstprod/hsc/ssp_pdr1/repro_1/utils/ingestImages/_parent/rawDownload` 

**Warning** : avoid to run a `ls` in this directory, there are >800 000 files and it will take a while...

As in the previous demo, we first setup the stack

```
source /sps/lsst/software/lsst_distrib/w_2017_38/loadLSST.bash
setup pipe_tasks
setup obs_subaru
```

We need to setup a couple more packages

```
setup meas_extensions_photometryKron
setup meas_extensions_convolved
```

Then we setup a directory which will contains links to input files (images, calibration, reference catalogs)

```
mkdir input
cd input
```

We create a `_mapper` file which will tell the stack which instrument we are going to use:

```
cca001[0] cat > _mapper
lsst.obs.hsc.HscMapper
^D
```

Create links to calibration and external reference catalog:

```
cca001[130] ln -s /sps/lsst/datasets/hsc/calib/20170105 CALIB
cca001[0] ln -s /sps/lsst/data/refcats/htm/htm_baseline ref_cats
cca001[0] ls -l
total 0
lrwxrwxrwx 1 boutigny lsst 37 Sep 26 18:31 CALIB -> /sps/lsst/datasets/hsc/calib/20170105
-rw-r--r-- 1 boutigny lsst 23 Sep 26 18:30 _mapper
lrwxrwxrwx 1 boutigny lsst 39 Sep 26 18:31 ref_cats -> /sps/lsst/data/refcats/htm/htm_baseline
```

All the stack I/O as well as some basic metadata are handled through a component called `butler`  

The first step is to ingest the raw image into the `butler` structure using the `ingestImages.py` command

```
cca010[0] cd ..
ca010[0] pwd
/sps/lsst/dev/boutigny/demo

cca010[0] ingestImages.py input /sps/lsst/dev/lsstprod/hsc/ssp_pdr1/repro_1/utils/ingestImages/_parent/rawDownload/HSCA04098410.fits
root INFO: Loading config overrride file '/sps/lsst/software/lsst_distrib/w_2017_38/stack/miniconda
root INFO: Loading config overrride file '/sps/lsst/software/lsst_distrib/w_2017_38/stack/miniconda3-4.3.21-10a4fa6/Linux64/obs_subaru/13.0-48-gb7c7029d/config/ingest.py'
CameraMapper INFO: Loading Posix exposure registry from /sps/lsst/dev/boutigny/demo/input
CameraMapper INFO: Loading calib registry from /sps/lsst/dev/boutigny/demo/input/CALIB/calibRegistry.sqlite3
ingest INFO: /sps/lsst/dev/lsstprod/hsc/ssp_pdr1/repro_1/utils/ingestImages/_parent/rawDownload/HSCA04098410.fits --<link>--> /sps/lsst/dev/boutigny/demo/input/SSP_UDEEP_SXDS/2015-10-10/01378/HSC-R/HSC-0040984-055.fits
```

This has simply created a link between the raw image in `/sps/lsst/dev/lsstprod/hsc/ssp_pdr1/repro_1/utils/ingestImages/_parent/rawDownload/HSCA04098410.fits` and `input/SSP_UDEEP_SXDS/2015-10-10/01378/HSC-R/HSC-0040984-055.fits`

Our input directory is now containing the following:

```
cca001[0] ls input
CALIB  SSP_UDEEP_SXDS  _mapper  ref_cats  registry.sqlite3
```

- We have one new directory: `SSP_UDEEP_SXDS` which contains the link to the raw image
- and an sqlite database containing 2 tables with some metadata

```
cca001[0] sqlite3 input/registry.sqlite3 
SQLite version 3.13.0 2016-05-18 10:57:30
Enter ".help" for usage hints.    
sqlite> .tables
raw        raw_visit

sqlite> select * from raw;
1|SSP_UDEEP_SXDS|40984|55|1378|HSC-R|o15426|2015-10-10|2015-10-10|300.0|0.0|1|-100.65|20140306.cfg|HSCA04098410|HSCA04098400|OBJECT

sqlite> select * from raw_visit;
40984|SSP_UDEEP_SXDS|HSC-R|2015-10-10|2015-10-10
```

These metadata are necessary for the butler they can be used to retrieve some missing informations to locate the files. They can also be used to identify which data are available in the butler

Looking what is under SSP_UDEEP_SXDS:

```
cca001[0] cd input/SSP_UDEEP_SXDS/
cca001[0] tree
.
`-- 2015-10-10
    `-- 01378
        `-- HSC-R
            `-- HSC-0040984-055.fits -> /sps/lsst/dev/lsstprod/hsc/ssp_pdr1/repro_1/utils/ingestImages/_parent/rawDownload/HSCA04098410.fits

3 directories, 1 file
```

In the HSC jargon:

- `SSP_UDEEP_SXDS` is the `field`
- `2015-10-10` is the observation date or `dateObs`
- `01378` is the `pointing`
- `HSC-R` is the `filter`
- `0040984` is the `visit`
- `055` is the `ccd` number


The command to make a single CCD image reduction is `processCcd.py`

```
cca001[0] processCcd.py input --output output --id visit=40984 ccd=55
```

See the output file `stack_output_example.txt` 

We see that a new `output` directory has been created. It contains the calibrated exposure corresponding to the input image as well as some catalogs

```
cca001[0] cd output
cca001[0] ls
01378  config  repositoryCfg.yaml  schema
```

- The `config` directory keeps track of the configuration parameters and the stack version used to run the task. The stack will prevent a task to be re-run with an incompatible set of parameters. The stack can be forced to run anyway even with different configuration files with the `--clobber-config` option

- The `schema`directory keeps track of the catalog schemas

- `repositoryCfg.yaml` is an internal configuration file for the butler


```
cca001[0] tree
.
|-- 01378
|   `-- HSC-R
|       |-- corr
|       |   |-- BKGD-0040984-055.fits
|       |   `-- CORR-0040984-055.fits
|       |-- output
|       |   |-- ICSRC-0040984-055.fits
|       |   |-- SRC-0040984-055.fits
|       |   |-- SRCMATCH-0040984-055.fits
|       |   `-- SRCMATCHFULL-0040984-055.fits
|       |-- processCcd_metadata
|       |   `-- 0040984-055.boost
|       `-- thumbs
|           |-- flattened-0040984-055.png
|           `-- oss-0040984-055.png
|-- config
|   |-- packages.pickle
|   `-- processCcd.py
|-- repositoryCfg.yaml
`-- schema
    |-- icSrc.fits
    `-- src.fits

8 directories, 14 files
```

The 2 most important files are:
- the calibrated exposure - CORR-0040984-055.fits - This is a fits image with 3 planes: 
  - image
  - masks 
  - variance
- the source catalog - SRC-0040984-055.fits